In [78]:
# imports
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import random
import time


In [79]:
# Function definitions for read/write html
def read_html(path): 
    with open(path, 'rb') as f: 
        return f.read()

def write_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [80]:
# Configure Selenium to use headless mode (optional)
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

ua = UserAgent()
user_agent = ua.random
chrome_options.add_argument(f"user-agent={user_agent}")

# Path to your ChromeDriver
chromedriver_path = '/path/to/chromedriver'

# Initialize WebDriver
service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

# Define the URL
url = 'https://www.reddit.com/r/AmItheAsshole/'

# Open the URL
driver.get(url)

# Scroll down to load more articles
SCROLL_PAUSE_TIME = random.uniform(2, 5)
last_height = driver.execute_script("return document.body.scrollHeight")

i = 0
while (i < 5):
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page
    time.sleep(SCROLL_PAUSE_TIME + random.uniform(0.5, 1.5))

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    i += 1
    
    # Random mouse movement to simulate human behavior
    action = ActionChains(driver)
    action.move_by_offset(random.randint(0, 100), random.randint(0, 100)).perform()
    
# Save the page source after loading more articles
raw_html_path = 'data/reddit-html.txt'
with open(raw_html_path, 'w', encoding='utf-8') as file:
    file.write(driver.page_source)

# Close the browser
driver.quit()

# Now you can parse the saved HTML as before
from bs4 import BeautifulSoup

with open(raw_html_path, 'r', encoding='utf-8') as file:
    raw_html = file.read()

soup = BeautifulSoup(raw_html, 'html.parser')
articles = soup.find_all('article', class_='w-full m-0')

# Print the number of articles found
print(f"Number of articles found: {len(articles)}")

'''
# Define website url and save html data 
url = 'https://www.reddit.com/r/AmItheAsshole/'
raw_html_path = 'data/reddit-html.txt'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}
req = requests.get(url, headers=headers)

write_html(req.content, raw_html_path)

# Check to see if data was written properly 
raw_html = read_html(raw_html_path)
#print(raw_html)
'''

Number of articles found: 50


'\n# Define website url and save html data \nurl = \'https://www.reddit.com/r/AmItheAsshole/\'\nraw_html_path = \'data/reddit-html.txt\'\nheaders = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}\nreq = requests.get(url, headers=headers)\n\nwrite_html(req.content, raw_html_path)\n\n# Check to see if data was written properly \nraw_html = read_html(raw_html_path)\n#print(raw_html)\n'

In [67]:
# Parse raw HTML to extract text
#soup = BeautifulSoup(req.content, 'html.parser')
#articles = soup.find_all('article', class_='w-full m-0') #TODO: Populate articles with more articles (selenium)

In [81]:
links = []

for article in articles: 
    shreddit_post = article.find('shreddit-post')
    links.append(shreddit_post.get('content-href'))

with open('data/reddit-links.txt', 'w', encoding='utf-8') as f: 
    for link in links:
        f.write(link + '\n')
        
        
print(links)

['https://www.reddit.com/r/AmItheAsshole/comments/1dk24fp/aita_for_cooking_meat_at_night_when_my_upstairs/', 'https://www.reddit.com/r/AmItheAsshole/comments/1dk1oim/aita_for_yelling_at_my_drunk_friend/', 'https://www.reddit.com/r/AmItheAsshole/comments/1diqid8/aita_for_walking_away_from_my_fils_wife_after_she/', 'https://www.reddit.com/r/AmItheAsshole/comments/1dk1ahi/aita_for_changing_my_plans/', 'https://www.reddit.com/r/AmItheAsshole/comments/1djs5r5/aita_for_not_taking_care_of_my_sisters_cat_when/', 'https://www.reddit.com/r/AmItheAsshole/comments/1dk3jwx/aita_for_telling_my_husband_he_sounds_rude_when/', 'https://www.reddit.com/r/AmItheAsshole/comments/1djmncq/aita_for_not_respecting_my_roommates/', 'https://www.reddit.com/r/AmItheAsshole/comments/1djdrp6/aita_for_being_angry_that_my_parents_tell_my/', 'https://www.reddit.com/r/AmItheAsshole/comments/1dk377d/aita_for_calling_out_my_friend_after_she_lied_to/', 'https://www.reddit.com/r/AmItheAsshole/comments/1djszuw/aita_for_askin

In [82]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}
for index, link in enumerate(links): 
     aita_post_req = requests.get(link, headers=headers)
     file_path = f'data/posts-html/aita-post{index}-html.txt'
     write_html(aita_post_req.content, file_path)

In [83]:
directory_path = 'data/posts-html'
items = os.listdir(directory_path)
for index, item in enumerate(items): 
    aita_post_soup = BeautifulSoup(read_html(directory_path + '/' + item), 'html.parser')
    post_container = aita_post_soup.find('div', class_='text-neutral-content')
    div1 = post_container.find('div')
    div2 = div1.find('div')
    p_elements = div2.find_all('p')
    post_text = '\n\n'.join(p.get_text(strip=True) for p in p_elements) # Concatenate text
    file_path = f'data/posts/aita-post{index}.txt'
    with open(file_path, 'w', encoding='utf-8') as f: 
        f.write(post_text)


In [71]:
'''
# Get text from post url
aita_post_soup = BeautifulSoup(aita_post_req.content, 'html.parser')
post_container = aita_post_soup.find('div', class_='text-neutral-content')
div1 = post_container.find('div')
div2 = div1.find('div')
p_elements = div2.find_all('p')
post_text = '\n\n'.join(p.get_text(strip=True) for p in p_elements) # Concatenate text

with open('data/post_text.txt', 'w', encoding='utf-8') as f:
        f.write(post_text)
          
print(post_text)
'''

"\n# Get text from post url\naita_post_soup = BeautifulSoup(aita_post_req.content, 'html.parser')\npost_container = aita_post_soup.find('div', class_='text-neutral-content')\ndiv1 = post_container.find('div')\ndiv2 = div1.find('div')\np_elements = div2.find_all('p')\npost_text = '\n\n'.join(p.get_text(strip=True) for p in p_elements) # Concatenate text\n\nwith open('data/post_text.txt', 'w', encoding='utf-8') as f:\n        f.write(post_text)\n          \nprint(post_text)\n"

In [72]:
'''
# New fetch request on AITA post url
aita_post_req = requests.get(links[0], headers=headers)
write_html(aita_post_req.content, 'data/aita-post-html.txt')
html = read_html('data/aita-post-html.txt')
print(html)
'''

"\n# New fetch request on AITA post url\naita_post_req = requests.get(links[0], headers=headers)\nwrite_html(aita_post_req.content, 'data/aita-post-html.txt')\nhtml = read_html('data/aita-post-html.txt')\nprint(html)\n"